<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [1]:
import csv
from collections import namedtuple   # Convenient to store the data rows

DATA_FILE = './data/chipotle.tsv'

In [2]:
with open(DATA_FILE, 'r') as file:
    file_nested_list = [row for row in csv.reader(file, delimiter='\t')]

### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [3]:
header = file_nested_list[0]
header

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

In [4]:
data = file_nested_list[1:]
data[:7]

[['1', '1', 'Chips and Fresh Tomato Salsa', 'NULL', '$2.39 '],
 ['1', '1', 'Izze', '[Clementine]', '$3.39 '],
 ['1', '1', 'Nantucket Nectar', '[Apple]', '$3.39 '],
 ['1', '1', 'Chips and Tomatillo-Green Chili Salsa', 'NULL', '$2.39 '],
 ['2',
  '2',
  'Chicken Bowl',
  '[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]',
  '$16.98 '],
 ['3',
  '1',
  'Chicken Bowl',
  '[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sour Cream, Guacamole, Lettuce]]',
  '$10.98 '],
 ['3', '1', 'Side of Chips', 'NULL', '$1.69 ']]

In [5]:
# Order = namedtuple('Order', 'order_id quantity item_name choice_description item_price')

---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [6]:
arr = [1, 1, 2, 3, 4, 5, 5, 1]
print( set(arr) )

{1, 2, 3, 4, 5}


In [7]:
num_orders = len(set([ row[0] for row in data ]))
num_orders

1834

In [8]:
# quantity already factored into item_price
price_list = []
for item in data:
    price_list.append( float( item[4].replace('$','') ) )
price_list[:6]

[2.39, 3.39, 3.39, 2.39, 16.98, 10.98]

In [18]:
price = [ float(row[4][1:-1]) for row in data ]
price[:6]

[2.39, 3.39, 3.39, 2.39, 16.98, 10.98]

In [10]:
round( sum(price_list) / num_orders , 2)

18.81

### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [11]:
data[18][2]

'Canned Soda'

In [12]:
data[18][2] == 'Canned Soda'

True

In [13]:
str(data[18][2]) is 'Canned Soda'

False

In [14]:
unique_sodas =[]
for row in data:
    if 'Canned' in row[2]:
        soda = str(row[3]).replace('[','').replace(']','')
        if soda not in unique_sodas:
            unique_sodas.append(soda)
unique_sodas

['Sprite',
 'Dr. Pepper',
 'Mountain Dew',
 'Diet Dr. Pepper',
 'Coca Cola',
 'Diet Coke',
 'Coke',
 'Lemonade',
 'Nestea']

---

## Advanced Level


### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


In [15]:
count = []
for item in data:
    if 'Burrito' in item[2]:
        toppings = item[3].replace('[','').replace(']','')
        toppings = toppings.split(',')
        count.append( len( toppings ))
print( sum(count) / len(count) )

5.395051194539249


### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

In [16]:
from collections import defaultdict

In [17]:
chip_count = defaultdict(int)
for item in data:
    if 'Chips' in item[2]:
        chip_count[ item[2] ] +=  int(item[1])
chip_count

defaultdict(int,
            {'Chips': 230,
             'Chips and Fresh Tomato Salsa': 130,
             'Chips and Guacamole': 506,
             'Chips and Mild Fresh Tomato Salsa': 1,
             'Chips and Roasted Chili Corn Salsa': 23,
             'Chips and Roasted Chili-Corn Salsa': 18,
             'Chips and Tomatillo Green Chili Salsa': 45,
             'Chips and Tomatillo Red Chili Salsa': 50,
             'Chips and Tomatillo-Green Chili Salsa': 33,
             'Chips and Tomatillo-Red Chili Salsa': 25,
             'Side of Chips': 110})

---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!
